# Delovni tok s človekom v zanki z Microsoft Agent Framework

## 🎯 Cilji učenja

V tem zvezku se boste naučili, kako implementirati delovne tokove **človek v zanki** z uporabo `RequestInfoExecutor` v Microsoft Agent Framework. Ta zmogljiv vzorec omogoča zaustavitev AI delovnih tokov za pridobivanje človeškega vnosa, kar naredi vaše agente interaktivne in daje ljudem nadzor nad ključnimi odločitvami.

## 🔄 Kaj je človek v zanki?

**Človek v zanki (HITL)** je oblikovalski vzorec, kjer AI agenti zaustavijo izvajanje, da bi zahtevali človeški vnos, preden nadaljujejo. To je ključno za:

- ✅ **Ključne odločitve** - Pridobite človeško odobritev pred sprejemanjem pomembnih ukrepov
- ✅ **Dvoumne situacije** - Dovolite ljudem, da pojasnijo, ko je AI negotov
- ✅ **Uporabniške preference** - Vprašajte uporabnike, naj izberejo med več možnostmi
- ✅ **Skladnost in varnost** - Zagotovite človeški nadzor za regulirane operacije
- ✅ **Interaktivne izkušnje** - Ustvarite pogovorne agente, ki se odzivajo na uporabniški vnos

## 🏗️ Kako deluje v Microsoft Agent Framework

Okvir ponuja tri ključne komponente za HITL:

1. **`RequestInfoExecutor`** - Poseben izvrševalec, ki zaustavi delovni tok in odda `RequestInfoEvent`
2. **`RequestInfoMessage`** - Osnovni razred za tipizirane zahteve, poslane ljudem
3. **`RequestResponse`** - Povezuje človeške odgovore z originalnimi zahtevami z uporabo `request_id`

**Vzorec delovnega toka:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Naš primer: Rezervacija hotela z uporabniško potrditvijo

Nadgradili bomo pogojni delovni tok z dodajanjem človeške potrditve **preden** predlagamo alternativne destinacije:

1. Uporabnik zahteva destinacijo (npr. "Pariz")
2. `availability_agent` preveri, ali so sobe na voljo
3. **Če ni sob** → `confirmation_agent` vpraša "Ali želite videti alternative?"
4. Delovni tok **se zaustavi** z uporabo `RequestInfoExecutor`
5. **Človek odgovori** "da" ali "ne" prek vnosa v konzolo
6. `decision_manager` usmeri glede na odgovor:
   - **Da** → Prikaži alternativne destinacije
   - **Ne** → Prekliči zahtevo za rezervacijo
7. Prikaži končni rezultat

To prikazuje, kako dati uporabnikom nadzor nad predlogi agenta!

---

Začnimo! 🚀


## Korak 1: Uvoz potrebnih knjižnic

Uvozimo standardne komponente Agent Frameworka ter **razrede, specifične za vključitev človeka v proces**:
- `RequestInfoExecutor` - Izvajalec, ki zaustavi potek dela za človeški vnos
- `RequestInfoEvent` - Dogodek, ki se sproži, ko je potreben človeški vnos
- `RequestInfoMessage` - Osnovni razred za tipizirane zahteve podatkov
- `RequestResponse` - Povezuje človeške odgovore z zahtevami
- `WorkflowOutputEvent` - Dogodek za zaznavanje izhodov poteka dela


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Korak 2: Določite Pydantic modele za strukturirane izhode

Ti modeli določajo **shemo**, ki jo bodo agenti vrnili. Ohranimo vse modele iz pogojnega delovnega toka in dodamo:

**Novo za vključitev človeka v proces:**
- `HumanFeedbackRequest` - Podrazred `RequestInfoMessage`, ki določa podatke zahteve, poslane ljudem
  - Vsebuje `prompt` (vprašanje za postavitev) in `destination` (kontekst o nedosegljivem mestu)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Korak 3: Ustvarite orodje za rezervacijo hotela

Isto orodje iz pogojnega delovnega toka - preveri, ali so sobe na voljo na ciljni lokaciji.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Korak 4: Določite funkcije pogojev za usmerjanje

Za naš delovni proces s človeškim posredovanjem potrebujemo **štiri funkcije pogojev**:

**Iz pogojnega delovnega procesa:**
1. `has_availability_condition` - Usmerja, ko so hoteli NA VOLJO
2. `no_availability_condition` - Usmerja, ko hoteli NISO NA VOLJO

**Novo za delovni proces s človeškim posredovanjem:**
3. `user_wants_alternatives_condition` - Usmerja, ko uporabnik reče "da" alternativam
4. `user_declines_alternatives_condition` - Usmerja, ko uporabnik reče "ne" alternativam


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Korak 5: Ustvarite izvršitelja Decision Manager

To je **jedro vzorca človek-v-zanki**! `DecisionManager` je prilagojen `Executor`, ki:

1. **Prejema povratne informacije od ljudi** prek objektov `RequestResponse`
2. **Obdeluje uporabnikovo odločitev** (da/ne)
3. **Usmerja potek dela** z pošiljanjem sporočil ustreznim agentom

Ključne značilnosti:
- Uporablja dekorator `@handler` za izpostavitev metod kot korakov poteka dela
- Prejema `RequestResponse[HumanFeedbackRequest, str]`, ki vsebuje tako prvotno zahtevo kot uporabnikov odgovor
- Generira preprosta sporočila "da" ali "ne", ki sprožijo naše funkcije pogojev


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Korak 6: Ustvarite prilagojenega izvajalca prikaza

Isti izvajalec prikaza iz pogojnega delovnega toka - prikaže končne rezultate kot izhod delovnega toka.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Korak 7: Naložite okoljske spremenljivke

Konfigurirajte odjemalca LLM (GitHub Models, Azure OpenAI ali OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Korak 8: Ustvarite AI agente in izvajalce

Ustvarimo **šest komponent delovnega toka**:

**Agenti (oviti v AgentExecutor):**
1. **availability_agent** - Preveri razpoložljivost hotelov z uporabo orodja
2. **confirmation_agent** - 🆕 Pripravi zahtevo za človeško potrditev
3. **alternative_agent** - Predlaga alternativna mesta (ko uporabnik reče da)
4. **booking_agent** - Spodbuja rezervacijo (ko so sobe na voljo)
5. **cancellation_agent** - 🆕 Obravnava sporočilo o odpovedi (ko uporabnik reče ne)

**Posebni izvajalci:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, ki začasno ustavi delovni tok za človeški vnos
7. **decision_manager** - 🆕 Prilagojen izvajalec, ki usmerja glede na človeški odziv (že definiran zgoraj)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Korak 9: Zgradite delovni tok s človekom v zanki

Zdaj sestavimo graf delovnega toka z **pogojnim usmerjanjem**, ki vključuje pot s človekom v zanki:

**Struktura delovnega toka:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Ključne povezave:**
- `availability_agent → confirmation_agent` (ko ni sob)
- `confirmation_agent → prepare_human_request` (sprememba vrste)
- `prepare_human_request → request_info_executor` (premor za človeka)
- `request_info_executor → decision_manager` (vedno - zagotavlja RequestResponse)
- `decision_manager → alternative_agent` (ko uporabnik reče "da")
- `decision_manager → cancellation_agent` (ko uporabnik reče "ne")
- `availability_agent → booking_agent` (ko so sobe na voljo)
- Vse poti se končajo pri `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Korak 10: Izvedite testni primer 1 - Mesto BREZ razpoložljivosti (Pariz s človeško potrditvijo)

Ta test prikazuje **celoten cikel s človekom v zanki**:

1. Zahteva za hotel v Parizu
2. availability_agent preveri → Ni sob
3. confirmation_agent ustvari vprašanje za človeka
4. request_info_executor **začasno ustavi potek dela** in sproži `RequestInfoEvent`
5. **Aplikacija zazna dogodek in pozove uporabnika v konzoli**
6. Uporabnik vnese "da" ali "ne"
7. Aplikacija pošlje odgovor prek `send_responses_streaming()`
8. decision_manager usmeri glede na odgovor
9. Prikazan končni rezultat

**Ključni vzorec:**
- Uporabite `workflow.run_stream()` za prvo iteracijo
- Uporabite `workflow.send_responses_streaming(pending_responses)` za nadaljnje iteracije
- Poslušajte `RequestInfoEvent`, da zaznate, kdaj je potreben človeški vnos
- Poslušajte `WorkflowOutputEvent`, da zajamete končne rezultate


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Korak 11: Izvedite testni primer 2 - Mesto Z razpoložljivostjo (Stockholm - Brez potrebe po človeškem vnosu)

Ta test prikazuje **neposredno pot**, ko so sobe na voljo:

1. Zahteva za hotel v Stockholmu
2. availability_agent preveri → Sobe so na voljo ✅
3. booking_agent predlaga rezervacijo
4. display_result prikaže potrditev
5. **Brez potrebe po človeškem vnosu!**

Delovni tok popolnoma obide pot s človeškim posredovanjem, ko so sobe na voljo.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Ključne točke in najboljše prakse za vključevanje človeka v proces

### ✅ Kaj ste se naučili:

#### 1. **Vzorec RequestInfoExecutor**
Vzorec vključevanja človeka v proces v Microsoft Agent Framework uporablja tri ključne komponente:
- `RequestInfoExecutor` - Ustavi potek dela in sproži dogodke
- `RequestInfoMessage` - Osnovni razred za tipizirane zahteve (podrazredite to!)
- `RequestResponse` - Povezuje odgovore ljudi z izvirnimi zahtevami

**Ključna razumevanja:**
- `RequestInfoExecutor` NE zbira vhodnih podatkov sam - le ustavi potek dela
- Vaša aplikacijska koda mora poslušati `RequestInfoEvent` in zbrati vhodne podatke
- Poklicati morate `send_responses_streaming()` z slovarjem, ki povezuje `request_id` z uporabnikovim odgovorom

#### 2. **Vzorec pretočne izvedbe**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Dogodkovno usmerjena arhitektura**
Poslušajte specifične dogodke za nadzor poteka dela:
- `RequestInfoEvent` - Potrebni so vhodni podatki od človeka (potek dela je ustavljen)
- `WorkflowOutputEvent` - Končni rezultat je na voljo (potek dela zaključen)
- `WorkflowStatusEvent` - Spremembe stanja (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS itd.)

#### 4. **Prilagojeni izvršitelji z @handler**
`DecisionManager` prikazuje, kako ustvariti izvršitelje, ki:
- Uporabljajo dekorator `@handler` za izpostavitev metod kot korakov poteka dela
- Prejemajo tipizirana sporočila (npr. `RequestResponse[HumanFeedbackRequest, str]`)
- Usmerjajo potek dela z pošiljanjem sporočil drugim izvršiteljem
- Dostopajo do konteksta prek `WorkflowContext`

#### 5. **Pogojno usmerjanje z odločitvami ljudi**
Lahko ustvarite funkcije pogojev, ki ocenjujejo odgovore ljudi:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Resnične aplikacije:

1. **Delovni tokovi odobritve**
   - Pridobite odobritev vodje pred obdelavo poročil o stroških
   - Zahtevajte pregled človeka pred pošiljanjem samodejnih e-poštnih sporočil
   - Potrdite transakcije z visoko vrednostjo pred izvedbo

2. **Moderiranje vsebine**
   - Označite vprašljivo vsebino za pregled človeka
   - Prosite moderatorje, da sprejmejo končno odločitev v mejnih primerih
   - Eskalirajte na ljudi, ko je zaupanje AI nizko

3. **Pomoč strankam**
   - Naj AI samodejno obravnava rutinska vprašanja
   - Eskalirajte kompleksne težave človeškim agentom
   - Vprašajte stranko, ali želi govoriti s človekom

4. **Obdelava podatkov**
   - Prosite ljudi, da rešijo dvoumne vnose podatkov
   - Potrdite interpretacije AI nejasnih dokumentov
   - Dovolite uporabnikom, da izberejo med več veljavnimi interpretacijami

5. **Sistemi, kritični za varnost**
   - Zahtevajte potrditev človeka pred nepopravljivimi dejanji
   - Pridobite odobritev pred dostopom do občutljivih podatkov
   - Potrdite odločitve v reguliranih industrijah (zdravstvo, finance)

6. **Interaktivni agenti**
   - Zgradite pogovorne bote, ki postavljajo dodatna vprašanja
   - Ustvarite čarovnike, ki vodijo uporabnike skozi kompleksne procese
   - Oblikujte agente, ki sodelujejo s človekom korak za korakom

### 🔄 Primerjava: Z in brez vključevanja človeka v proces

| Funkcija | Pogojni potek dela | Potek dela z vključevanjem človeka |
|----------|--------------------|-----------------------------------|
| **Izvedba** | Enkratni `workflow.run()` | Zanka z `run_stream()` + `send_responses_streaming()` |
| **Vhod uporabnika** | Brez (popolnoma avtomatizirano) | Interaktivni pozivi prek `input()` ali UI |
| **Komponente** | Agenti + izvršitelji | + RequestInfoExecutor + DecisionManager |
| **Dogodki** | Samo AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent itd. |
| **Ustavitev** | Brez ustavitve | Potek dela se ustavi pri RequestInfoExecutor |
| **Nadzor človeka** | Brez nadzora človeka | Ljudje sprejemajo ključne odločitve |
| **Uporaba** | Avtomatizacija | Sodelovanje in nadzor |

### 🚀 Napredni vzorci:

#### Več točk odločanja ljudi
V istem poteku dela lahko imate več vozlišč `RequestInfoExecutor`:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Upravljanje časovnih omejitev
Implementirajte časovne omejitve za odgovore ljudi:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integracija z bogatimi uporabniškimi vmesniki
Namesto `input()` integrirajte z web UI, Slack, Teams itd.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Pogojno vključevanje človeka v proces
Zahtevajte vhod človeka le v specifičnih situacijah:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Najboljše prakse:

1. **Vedno podrazredite RequestInfoMessage**
   - Zagotavlja varnost tipov in validacijo
   - Omogoča bogat kontekst za upodabljanje UI
   - Jasno izraža namen vsake vrste zahteve

2. **Uporabljajte opisne pozive**
   - Vključite kontekst o tem, kaj sprašujete
   - Pojasnite posledice vsake izbire
   - Naj bodo vprašanja preprosta in jasna

3. **Obravnavajte nepričakovane vnose**
   - Validirajte odgovore uporabnikov
   - Zagotovite privzete vrednosti za neveljavne vnose
   - Dajte jasna sporočila o napakah

4. **Sledite ID-jem zahtev**
   - Uporabite korelacijo med request_id in odgovori
   - Ne poskušajte ročno upravljati stanja

5. **Oblikujte za neblokiranje**
   - Ne blokirajte niti, ki čakajo na vhod
   - Uporabljajte asinhrone vzorce povsod
   - Podprite sočasne primere poteka dela

### 📚 Povezani koncepti:

- **Agent Middleware** - Prekinitev klicev agentov (prejšnji zvezek)
- **Upravljanje stanja poteka dela** - Ohranjanje stanja poteka dela med izvajanji
- **Sodelovanje več agentov** - Združite vključevanje človeka v proces z ekipami agentov
- **Dogodkovno usmerjene arhitekture** - Zgradite reaktivne sisteme z dogodki

---

### 🎓 Čestitke!

Obvladali ste poteke dela z vključevanjem človeka v proces z Microsoft Agent Framework! Zdaj veste, kako:
- ✅ Ustaviti poteke dela za zbiranje vhodnih podatkov od človeka
- ✅ Uporabljati RequestInfoExecutor in RequestInfoMessage
- ✅ Upravljati pretočno izvedbo z dogodki
- ✅ Ustvariti prilagojene izvršitelje z @handler
- ✅ Usmerjati poteke dela na podlagi odločitev ljudi
- ✅ Zgraditi interaktivne AI agente, ki sodelujejo z ljudmi

**To je ključni vzorec za gradnjo zaupanja vrednih in nadzorovanih AI sistemov!** 🚀



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje AI [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem maternem jeziku naj se šteje za avtoritativni vir. Za ključne informacije priporočamo profesionalni človeški prevod. Ne odgovarjamo za morebitne nesporazume ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
